In [ ]:
import cv2
import gc
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from tqdm import tqdm_notebook as tqdm
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import multiprocessing
import warnings
from glob import glob
import os
import imageio
import time
warnings.filterwarnings("ignore")

In [ ]:
def read_xray(path):
    voi_lut = True
    fix_monochrome = True
    dicom = pydicom.read_file(path)
    basename = os.path.basename(path).split('.')[0]
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return basename, data

In [ ]:
cpu_count = multiprocessing.cpu_count()
image_paths = "../input/vinbigdata-chest-xray-abnormalities-detection/train"

**Process 100 images**

In [ ]:
start = time.time()
image_paths = glob(os.path.join(image_paths, '*.dicom'))[:10]
try:
    pool = multiprocessing.Pool(processes = cpu_count)
    for basename, image in pool.map(read_xray, image_paths):
        imageio.imwrite(basename + '.png', image)
finally:
    pool.close()
    pool.join()

print("Time Execution : {}".format(time.time() - start))